In [1]:
!pip install gspread
!pip install google-auth


In [2]:
import sqlite3

In [5]:
import sqlite3
import gspread
from google.oauth2.service_account import Credentials

# Google Sheets setup
SCOPE = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('/content/credentials.json', scopes=SCOPE)
client = gspread.authorize(creds)

sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/187uudwa8AuEx_2XFm2RT29SKCj7l1KRBEF2ooVbVYBg/edit#gid=0')
worksheet = sheet.get_worksheet(0)

# Database functions with context manager
def connect_to_db():
    """Create a connection to the SQLite database."""
    return sqlite3.connect('sync_db.db')

def create_table():
    """Create the table if it doesn't already exist."""
    with connect_to_db() as conn:
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS sync_table (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                email TEXT,
                phone_number TEXT
            )
        ''')
        conn.commit()

def sync_google_sheets_to_sqlite():
    """Sync data from Google Sheets to the SQLite database."""
    with connect_to_db() as conn:
        cursor = conn.cursor()
        rows = worksheet.get_all_values()[1:]  # Skip the header

        if not all(len(row) == 3 for row in rows):
            raise ValueError("Data rows do not match the expected format of 3 columns.")

        # Clear the SQLite table
        cursor.execute("DELETE FROM sync_table")

        # Insert data from Google Sheets into SQLite
        cursor.executemany("INSERT INTO sync_table (name, email, phone_number) VALUES (?, ?, ?)", rows)

        conn.commit()
    print("Google Sheets data synced to SQLite successfully!")

def sync_sqlite_to_google_sheets():
    """Sync data from the SQLite database to Google Sheets."""
    with connect_to_db() as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT name, email, phone_number FROM sync_table")
        rows = cursor.fetchall()

    # Clear existing data in the Google Sheet and insert headers
    worksheet.clear()  # Clears all the content in the sheet
    worksheet.append_row(["Name", "Email", "Phone Number"])

    # Insert SQLite rows back to Google Sheets
    for row in rows:
        worksheet.append_row(list(row))

    print("SQLite data synced to Google Sheets successfully!")

def sync_bidirectional():
    """Perform bidirectional synchronization between Google Sheets and SQLite."""
    sync_google_sheets_to_sqlite()
    sync_sqlite_to_google_sheets()

# Create the table if it doesn't exist
create_table()

# Perform the bidirectional sync



In [11]:
sync_google_sheets_to_sqlite()

Google Sheets data synced to SQLite successfully!


In [16]:
sync_sqlite_to_google_sheets()

SQLite data synced to Google Sheets successfully!


In [15]:
def show_data_from_db():
    conn = connect_to_db()
    cursor = conn.cursor()

    # Query to fetch all data from sync_table
    cursor.execute("SELECT * FROM sync_table")
    rows = cursor.fetchall()

    # Print the results
    print("Data from SQLite database:")
    for row in rows:
        print(row)

    cursor.close()
    conn.close()
show_data_from_db()

Data from SQLite database:
(4, 'Alice Johnson', 'alice.johnson@example.com', '+1234567890')
(5, 'Bob Smith', 'bob.smith@example.com', '+0987654321')
(6, 'Charlie Brown', 'charlie.brown@example.com', '+1122334455')


In [14]:
def insert_sample_records():
    """Insert some sample records into the SQLite database."""
    sample_records = [
        ('Alice Johnson', 'alice.johnson@example.com', '+1234567890'),
        ('Bob Smith', 'bob.smith@example.com', '+0987654321'),
        ('Charlie Brown', 'charlie.brown@example.com', '+1122334455')
    ]

    with connect_to_db() as conn:
        cursor = conn.cursor()
        cursor.executemany("INSERT INTO sync_table (name, email, phone_number) VALUES (?, ?, ?)", sample_records)
        conn.commit()

    print("Sample records inserted into SQLite database successfully!")
insert_sample_records()

Sample records inserted into SQLite database successfully!
